<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH18_06_KLUE_BERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B0%9C%EC%B2%B4%EB%AA%85%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.2 MB/s eta 0:00:00


### 1. 데이터로드

In [2]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=844102be7d995d77350baab079957f371fff6a13afdee0dc44c9f4e25c47f1f6
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/\
tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20\
NLI)/dataset/ner_train_data.csv", filename="ner_train_data.csv")

urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/\
tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20\
NLI)/dataset/ner_test_data.csv", filename="ner_test_data.csv")

urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/\
tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20\
NLI)/dataset/ner_label.txt", filename="ner_label.txt")


('ner_label.txt', <http.client.HTTPMessage at 0x7fb929d57370>)

In [5]:
train_ner_df = pd.read_csv("ner_train_data.csv")
test_ner_df = pd.read_csv('ner_test_data.csv')

In [6]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [7]:
train_ner_df.shape, test_ner_df.shape

((81000, 2), (9000, 2))

In [8]:
# 훈련데이터와 테스트 데이터에서 문장과 레이블을 각각 분리해 저장
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]

In [9]:
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [10]:
print(train_data_sentence[2])
print(train_data_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


이 데이터는 형태소 단위가 아니라 어절단위로 개체명 인식레이블이 태깅되었다는 특징이 있음. ner_label.txt 파일을 읽어들여 개체명 태깅 정보의 종류를 확인해 봄

In [11]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개체명 태깅정보 :', labels)

개체명 태깅정보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


개체명 태깅정보를 저장한 labels 리스트로부터 각 개체명 태깅정보와 정수를 맵핑하는 딕셔너리를 만듬. tag_to_index는 개체명 태깅 정보를 key, 정수를 value로 하는 딕셔너리이며 index_to_tag는 그 반대 형태를 가지는 딕셔너리임

In [12]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [13]:
print(tag_to_index)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}


In [14]:
print(index_to_tag)

{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [15]:
tag_size = len(tag_to_index)

In [16]:
tag_size

29

### 2. 전처리 예시

In [17]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

전처리 과정에 대한 이해

In [18]:
sent = train_data_sentence[1]
label = train_data_label[1]

print('문장:', sent)
print('레이블:', label)
print('레이블의 정수인코딩:', [tag_to_index[idx] for idx in label])
print('문장길이:', len(sent))
print('레이블 길이:', len(label))

문장: ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
레이블: ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수인코딩: [0, 1, 2, 0, 1, 0, 1]
문장길이: 7
레이블 길이: 7


In [19]:
# BERT 토크나이저 적용과정 이해
tokens = []
for one_word in sent:
  # 각 단어에 대해 서브워드로 분리
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
print('BERT 토크나이저 적용 후 문장:', tokens)
print('레이블 :', label)
print('레이블의 정수 인코딩:', [tag_to_index[idx] for idx in label])
print('문장 길이 :', len(tokens))
print('레이블 길이:', len(label))

BERT 토크나이저 적용 후 문장: ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩: [0, 1, 2, 0, 1, 0, 1]
문장 길이 : 16
레이블 길이: 7


단어들이 서브워드로 분리되었기 때문에, 문장길이와 레이블 길이가 달라짐. 여기에서는 '##'가 붙어 있는 서브워드에는 레이블을 주지 않는 방식으로 정수 인코딩 수행.

In [20]:
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]] + [-100] * (len(subword_tokens) - 1))  # 길이 만큼  -100 

print('토큰화 후 문장 :', tokens)
print('레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩: ', labels_ids)
print('레이블의 길이 :', len(labels_ids))

토큰화 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩:  [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
레이블의 길이 : 16


'한동수가'에 PER-1가 부여되어 있었으나, 서브워드 토큰화 과정에서 ['한동', '##수', '##가']로 분리되었고, 첫번째 서브워드인 '한동'에 PER-1에 해당하는 정수인 2를 부여하지만 그 뒤의 서브워드들인 '##수', '##가'에 대해 -100을 부여함.


여기에서는 -100을 토큰 [PAD]로 사용. 레이블에 대해서는 문장의 길이를 맞추는 패딩을 진행할 때도 -100을 사용함. 그리고 이후 손실함수에서 -100을 무시하도록 추가처리 진행할 것임


다음은 세그먼트 인코딩, 어텐션 마스크까지 생성하는 함수 구현

### 3. 전처리

In [21]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, 
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    tokens = []
    labels_ids = []
    for one_word, label_token in zip(example, label):
      # 하나의 단어에 대해 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채움
      labels_ids.extend([tag_to_index[label_token]] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

      # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len-2 길이로 변환
      sepcial_tokens_count = 2
    if len(tokens) > max_seq_len - sepcial_tokens_count:
      tokens = tokens[:(max_seq_len - sepcial_tokens_count)]  # 끝 부분은 잘라냄
      labels_ids = labels_ids[:(max_seq_len - sepcial_tokens_count)] # 마찬가지

    # [SEP]을 추가하는 코드
    # 1. 토큰화 결과의 맨 뒷 부분에 [SEP] 토큰 추가
    # 2. 레이블에도 맨뒷 부분에 -100 추가
    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]

    # [CLS]를 추가하는 코드
    # 1. 토큰화 결과의 맨 앞 부분에 [CLS] 토큰 추가
    # 2. 레이블에도 맨앞 부분에 -100 추가
    tokens = [cls_token] + tokens
    labels_ids = [pad_token_id_for_label] + labels_ids

    # 정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

    # 어텐션 마스크 생성
    attention_mask = [1] * len(input_id)

    # 정수 인코딩에 추가할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)

    # 정수 인코딩, 어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

    # 세그먼트 인코딩
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레이블 패딩. 단, 이 경우에는 패딩토큰의 ID가 -100으로 함
    label = labels_ids + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, 'Error with input length {} vs {}'.format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, 'Error with attention mask length{} vs {}'.format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, 'Error with token type length {} vs {}'.format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, 'Error with label length {} vs {}'.format(len(label), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [22]:
# 훈련데이터와 테스트 데이터 전처리 진행
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:12<00:00, 737.12it/s] 


첫번째 샘플 확인!!

In [23]:
X_sample, y_sample = convert_examples_to_features(train_data_sentence[:1], train_data_label[:1], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 1/1 [00:00<00:00, 728.56it/s]


In [24]:
print('기존 원문:', train_data_sentence[0])
print('기존 레이블: ', train_data_label[0])
print('-' * 50)
print('토큰화 후 원문:', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토큰화 후 레이블: ', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])
print('-' * 50)
print('정수 인코딩 결과 :', X_train[0][0])
print('정수 인코딩 레이블 :', y_train[0])

기존 원문: ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기존 레이블:  ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토큰화 후 원문: ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [25]:
print('세그먼트 인코딩: ', X_train[2][0])
print('어텐션 마스크: ', X_train[1][0])

세그먼트 인코딩:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
어텐션 마스크:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 4. 모델링

텍스트 분류를 수행할 때는 BERT의 출력인 outputs에서 인덱스 1로 접근하였음. outputs[1]은 [CLS] 토큰에 접근하는 방법으로 Many-to-One 문제에 해당하는 텍스트 분류 문제를 풀때 사용함. 

여기에서는 모든 입력에 대해 출력을 수행해야 하는 Many-to-Many 문제에 해당하므로 outputs[0]을 사용하여 문제를 풀어야 함


outputs[0]과 연결되는 출력층에는 레이블 개수인 num_labels를 전달함. 이번에는 다중 클래스 분류 문제임에도 출력층에 소프트맥스 함수를 사용하지 않았는데, 이번에는 출력층에 소프트맥스함수를 누락시킨 후에 손실함수에서 이를 처리하도록 하는 방식을 사용하기 위함임

In [26]:
class TFBertForTokenClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForTokenClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels, 
                                            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), 
                                            name='classifier')
  
  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    all_output = outputs[0]
    prediction = self.classifier(all_output)
    return prediction

### 5. 손실함수에서 -100 레이블은 제외하기(예시)

레이블에서 -100인 경우 학습에 반영하지 않는 코드 구현... 


In [27]:
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15], [0.4, 0.5, 0.1]]]) # shape=(1, 4, 3)

In [28]:
# -100 구간과 아닌 구간을 나타내는 active_loss 만들기
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


In [29]:
shape_list(logits)

[1, 4, 3]

In [30]:
tf.reshape(logits, (-1, 3)) # labels과 동일한 차원으로 맞춤

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0.8 , 0.1 , 0.1 ],
       [0.06, 0.04, 0.9 ],
       [0.75, 0.1 , 0.15],
       [0.4 , 0.5 , 0.1 ]], dtype=float32)>

In [31]:
# active_loss를 활용하여 logits에서 -100 위치를 가지는 예측값은 제거함
reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
print(reduced_logits)  # 첫번째, 네번째 행이 삭제됨

tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


In [32]:
labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


### 6. 손실함수 구현하기

In [33]:
def compute_loss(labels, logits):
  # 다중 클래스 분류 문제에서 소프트맥스 함수 미사용시 from_logits=True로 설정 
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  # -100 값을 가진 정수에 대해 오차를 반영하지 않도록 labels 수정
  active_loss = tf.reshape(labels, (-1,)) != -100
  # activa_loss로부터 reduced_logits와 labels를 각각 얻음
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)

손실함수로는 다중 클래스 분류 문제이므로, 다중클래스 분류 문제에서 사용했던 SparseCategoricalCrossentropy를 사용함. 다만 -100 레이블에 대해 [PAD] 토큰으로 간주하여 loss를 구하지 않는 것으로 가정하고 전처리를 진행하였으므로 손실함수에서도 이를 반영해 주어야 함...


한가지 더 고려해야 할 점은 앞서 만든 TFBertForTOkenClassification 모델에서 출력층에 소프트맥스함수를 사용하지 않나 예측벡터의 총 합이 1이 되지 않은 상태라는 점임.

이 경우, SparseCategoricalCrossentropy에서 from_logits=True로 하면 예측값이 소프트맥스 함수를 통과하지 않았음을 고려하여 오차를 구함.



### 7. 학습

In [35]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [36]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

한 에포크가 끝날때마다 테스트 데이터에 대해 F1 score를 계산하여 학습이 과최적화 되는지 판단하고자 함. 이를 위해 tf.keras.callbacks.Callback 을 상속받아 커스텀 콜백 F1score를 구현함

In [39]:
class F1score(tf.keras.callbacks.Callback):
  def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test

  def sequences_to_tags(self, label_ids, pred_ids):
    label_list = []
    pred_list = []

    for i in range(0, len(label_ids)):
      label_tag = []
      pred_tag = []

      # 레이블의 값이 -100인 경우는 F1 score 계산 시에도 제외
      for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
        if label_index != -100:
          label_tag.append(index_to_tag[label_index])
          pred_tag.append(index_to_tag[pred_index])
      label_list.append(label_tag)
      pred_list.append(pred_tag)

    return label_list, pred_list

    # 에포크가 끝날때마다 실행되는 함수
  def on_epoch_end(self, epoch, logs={}):
    y_predicted = self.model.predict(self.X_test)
    y_predicted = np.argmax(y_predicted, axis=2)

    label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
    score = f1_score(label_list, pred_list, suffix=True)
    print(' - f1: {:04.2f}'.format(score * 100))
    print(classification_report(label_list, pred_list, suffix=True))

In [40]:
# batch_size=32, epochs=3
f1_score_report = F1score(X_test, y_test)
model.fit(X_train, y_train, epochs=3, batch_size=32, callbacks=[f1_score_report])

Epoch 1/3
   6/2532 [..............................] - ETA: 3:05 - loss: 0.0612

282/282 [==============================] - 18s 50ms/step
 - f1: 86.33
              precision    recall  f1-score   support

         AFW       0.58      0.66      0.62       394
         ANM       0.77      0.79      0.78       701
         CVL       0.83      0.85      0.84      5758
         DAT       0.92      0.93      0.93      2521
         EVT       0.76      0.76      0.76      1094
         FLD       0.69      0.57      0.62       228
         LOC       0.89      0.83      0.86      2126
         MAT       0.20      0.17      0.18        12
         NUM       0.93      0.92      0.92      5590
         ORG       0.88      0.88      0.88      4086
         PER       0.90      0.89      0.89      4426
         PLT       0.45      0.29      0.36        34
         TIM       0.84      0.94      0.89       314
         TRM       0.77      0.73      0.75      1964

   micro avg       0.86      0.86      0.86     29248
   macro avg       0.74      0.73      0.73     29248
weighted a

### 8. 예측

In [51]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer, 
                                                pad_token_id_for_segment=0, pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

  for example in tqdm(examples):
    tokens, label_mask = [], []
    for one_word in example:
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫번째 서브워드를 제외하고 모두 -100으로 채움
      label_mask.extend([0] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

      # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len-2 길이로 변환
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      label_mask = label_mask[:(max_seq_len - special_tokens_count)]

      # [SEP]를 추가하는 코드
      # 1. 토큰화 결과의 맨 뒷부분에 [SEP] 토큰 추가
      # 2. 레이블에도 맨뒷 부분에 -100 추가
    tokens += [sep_token]
    label_mask += [pad_token_id_for_label]

      # [CLS]를 추가하는 코드
      # 1. 토큰화 결과의 맨 앞부분에 [CLS] 토큰 추가
      # 2. 레이블에도 맨앞 두분에 -100 추가
    tokens = [cls_token] + tokens
    label_mask = [pad_token_id_for_label] + label_mask

      # 정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

      # 어텐션 마스크 생성
    attention_mask = [1] * len(input_id)

      # 정수 인코딩에 추가할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)

      # 정수 인코딩, 어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

      # 세그먼트 인코딩
    token_type_id = [pad_token_id_for_segment] * max_seq_len

      # 레이블 패딩
    label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
    assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    label_masks.append(label_mask)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  label_masks = np.asarray(label_masks, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), label_masks

In [52]:
# 샘플로 테스트
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 1008.00it/s]


In [53]:
print('기존 원문 :', test_data_sentence[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레이블 마스크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기존 원문 : ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [54]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis=2)

  pred_list = []
  result_list = []
  for i in range(0, len(label_masks)):
    pred_tag = []
    # 모델의 예측값 디코딩 과정
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])
    pred_list.append(pred_tag)
  
  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)
  return result_list

In [55]:
# 훈련데이터에 존재하지 않았던 임의의 두개 문장에 대해 개체명을 예측
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다.'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

test_samples = [sent1, sent2]

result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)
result_list

100%|██████████| 2/2 [00:00<00:00, 478.04it/s]


1/1 [==============================] - 1s 1s/step


[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다.', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]